# Tutorial para instalar QIIME en PATUNG

Tutorial para instalar QIIME en el servidor de PATUNG del LANCIS. En este tutorial vamos a instalar QIIME, a tranferir nuestros archivos a PATUNG y a generar el archivo manifiesto necesario para importar nuestras secuencias a QIIME.
Se asume que el usuario ya tiene cuenta y contraseña del servidor de PATUNG.
El usuario debe tener conocimiento de BASH para crear y mover archivos (no es cierto, pongo los comandos aquí pero si ayuda).

* El código que se presenta debe copiarse en el Bash de Linux. Nota: si no te gusta Linux (como a mi) todo el proceso se puede realizar en Windows 10 usando el subsistema Linux/Ubuntu sin usar máquina virtual. En caso de usar el subsitema de Linux en Windows hay que tomar en cuenta que se deben pasar los archivos a la carpeta home del subsitema Linux puesto que no se puede acceder a los archivos de Windows desde ahí. Para pasar archivos al subsitema linux hay que hacerlo en la carpeta con dirección similar a esta: 

C:\Users\chelo\AppData\Local\Packages\CanonicalGroupLimited.UbuntuonWindows_79rhkp1fndgsc\LocalState\rootfs\home\chelo

El primer paso es acceder a nuestra sesión en PATUNG e ingresar nuestra contraseña.

In [ ]:
ssh nombre_de_usuario@patung

## Empecemos instalando QIIME!

Para eso necesitamos instalar Miniconda. Vamos a descargar el instalador de Miniconda. Usemos el comando __```wget```__. El argumento del comando es la url del instalador.

In [ ]:
wget 'https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh'

Siempre que bajamos un archivo de internet hay que hacerlo ejecutable. Usemos el comando __```chmod```__. Sus argumentos son ```+x``` que significa que concedemos el permiso de ejecución y el nombre del archivo ejecutable.

In [ ]:
chmod +x Miniconda3-latest-Linux-x86_64.sh

Instalar Miniconda.

In [ ]:
./Miniconda3-latest-Linux-x86_64.sh

Presionar enter hasta que pida confirmación y escribir 'yes'.

Hay que salir de PATUNG

In [ ]:
exit

Luego entramos de nuevo a nuestra sesión de PATUNG y actualizamos Miniconda.

In [ ]:
conda update conda

Vamos a obtener el instalador de QIIME.

In [ ]:
wget https://data.qiime2.org/distro/core/qiime2-2018.8-py35-linux-conda.yml

Creamos un ambiente virtual donde vamos a usar QIIME.

In [ ]:
conda env create -n qiime2-2018.8 --file qiime2-2018.8-py35-linux-conda.yml

Probamos el ambiente virtual activándolo.

In [ ]:
source activate qiime2-2018.8

Comprobamos instalación de QIIME.

In [ ]:
qiime --help

Activamos la función de autocompletar los comandos de QIIME usando la tecla tab. Hay que hacerlo cada vez que activemos el ambiente virtual.

In [ ]:
source tab-qiime

Ya instalamos QIIME! Vamos a desactivar el ambiente virtual por el momento.

In [ ]:
source deactivate

Salimos de patung.

In [ ]:
exit

## Tranferencia de archivos a PATUNG (y viceversa)

Ahora necesitamos transferir nuestros archivos a PATUNG!

Para sincronizar nuestra carpeta de resultados usamos el comando __```rsync__```. Sus argumentos son:

* -azp; sirve para añadir compresión, una barra de transferencia y reanudar transferencias incompletas.
* Path de origen
* Path de destino

El proceso lo realizamos en nuestra PC local por lo que si estamos en PATUNG abrimos otra terminal local o salimos de PATUNG.

La primera vez que sincronizamos nuestros archivos, hay que crear nuestra carpeta del análisis y nos movemos ahí.

In [ ]:
mkdir metag_granulos_hidro

In [ ]:
cd metag_granulos_hidro

Aquí, la primera vez creamos la estrucutra de subcarpetas para nuestro análisis. EG: secuencias, executables, resultados, errores. No es necesario hacerlo en la terminal ya que podemos crearlas en el navegador de archivos de nuestro sistema operativo.

También, podemos hacerlo en nuestra terminal. Para esto, debemos estar dentro de la carpeta del análisis y usar el comando __```mkdir```__.

Usamos el comando __```pwd```__ para verificar que nos encontramos en nuestra carpeta del análisis.

In [ ]:
pwd

Una vez dentro de la carpeta del análisis creamos la estructura de nuestro folder del análisis.

In [ ]:
mkdir 1_secuencias 2_resultados 3_ejecutables 4_outs 5_logs 6_errores

Y copiamos nuestras secuencias fasta en el folder correspondiente.

In [ ]:
cd ~/secuencias

cp . ~/metag_granulos_hidro/1_secuencias

Luego usamos el comando __```rsync```__. Debes estar un nivel arriba de la carpeta del análisis. La primera vez vamos a copiar la carpeta creada en PAUTNG. Reemplaza con el nombre de tu carpeta de PATUNG en el siguiente ejemplo.

In [ ]:
sudo rsync -azP ./metag_granulos_hidro/1_secuencias anavarro@patung:/srv/home/anavarro/

Para las siguientes ocasiones también debes estar un nivel arriba de tu carpeta de análisis pero debes correr el comando incluyendo el nombre de tu carpeta del análisis en el destino. Este comando sólo agregará los archivo modificados o nuevos. Importante las diagonales al final del argumento de origen y destino para que suceda esto.

In [ ]:
sudo rsync -azP ./metag_granulos_hidro/ anavarro@patung:/srv/home/anavarro/metag_granulos_hidro/

Tienes que ingresar tu contraseña de PATUNG y listo! ya tienes tus archivos en PATUNG. Ingresa a PATUNG y verifícalo.

Una vez terminado nuestro análisis y para respaldar el trabajo que realicemos 
hay que transferir archivos de PATUNG a nuestra PC. También usamos el comando __```rsync```__. También en esta ocasión tenemos que estar un nivel arriba de la carpeta local del análisis.

In [ ]:
rsync -azP anavarro@patung:/srv/home/anavarro/metag_granulos_hidro .

Cada vez que pasemos archivos de esta forma hay que hacerlos executables y permitir su lectura. Usemos el siguiente comando.

In [ ]:
chmod +rwx -R ./metag_granulos_hidro

## Generar el archivo manifiesto.

Para usar QIIME vamos a necesitar nuestras secuencias en formato fasta (archivos R1 y R2) y el archivo manifiesto que indica dónde están (path) y su dirección en la cadena de DNA (reverse y forward).

En PATUNG, navegamos hasta la carpeta del análisis donde tenemos nuestras secuencias usando el comando cd.

In [ ]:
cd metag_granulos_hidro/1_secuencias

Revisamos esta carpeta y nuestras secuencias en fasta deben estar ahí.

In [ ]:
ls

Vamos a generar un archivo de texto que incluya el path de cada archivo fasta. Usamos el comando __```ls```__ con el argumento ```-l``` que indica que queremos los resultados en una lista. Luego va el path donde están nuestras secuencias. Finalmente usamos '>nombre_archivo.txt' para indicar que queremos los resultados en un archivo de texto.

In [ ]:
ls -1 /srv/home/anavarro/metag_granulos_hidro/1_secuencias/*_R1* /srv/home/anavarro/metag_granulos_hidro/1_secuencias/*_R2* > seqs_manifest.txt

Revisamos nuestro archivo de texto. Presionar ```q``` para salir de la vista previa.

In [ ]:
less seqs_manifest.txt

Volvemos a sincronizar nuestros archivos de PATUNG a nuestra PC local en otra terminal o saliendo de PATUNG. Esta vez __```rsync```__ sólo va a transferir los archivos modificados o nuevos (en este caso el archivo manifiesto). Una vez sincronizados nuestros archivos a nuestra PC abrimos el archivo manifiesto en Excel o nuestro software de hojas de cálculo favorito. Agregamos una primer columna donde va el nombre de la muestra. En la segunda dejamos el PATH y en la tercera ponemos 'forward' o 'reverse' según sea el caso. Ponemos una cabezera a nuestro archivo con los nombres para cada columna: 

* sample-id
* absolute-filepath
* direction

Guardamos el archivo como txt separado por comas. Importante que la extensión del archivo sea txt. Y sincronizamos nuestros archivos de vuelta a PATUNG.

Listo! ya tenemos todos los archivos para empezar a usar QIIME!